In [1]:
import os
# 【硬性要求】加载环境变量
from dotenv import load_dotenv
load_dotenv(override=True)

# 引入核心模块
from langchain_community.document_loaders import PyPDFLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# ==========================================
# 1. 索引阶段：数据源的热插拔 (Hot-swapping Data Sources)
# ==========================================
print("1. 正在初始化数据摄取...")

# 【原代码 - 已注释】使用 PDF 加载器
# loader = PyPDFLoader("data/handbook.pdf")

# 【新代码 - 无缝接入】使用网页加载器，读取 Llama 3 的最新新闻
URL = "https://www.theverge.com/2024/4/18/24133808/meta-ai-assistant-llama-3-chatgpt-openai-rival"
loader = WebBaseLoader(URL)

# 后续的所有 ETL 流程完全不用改！
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

print("2. 正在进行向量化并存入数据库...")
# 初始化 Embeddings 模型并建立向量数据库
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings)

# ==========================================
# 2. 检索与生成组件设置 (完全保持不变)
# ==========================================
retriever = vectorstore.as_retriever()

def format_docs(retrieved_docs):
    return "\n\n".join(doc.page_content for doc in retrieved_docs)
    
llm = ChatOpenAI(model="gpt-4o", temperature=0)

template = """SYSTEM: You are a question answer bot.
Be factual in your response.
Respond to the following question ONLY from the below context: 
{context}

If you don't know the answer, just say that you don't know.

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

# ==========================================
# 3. 组装 LCEL 管道 (完全保持不变)
# ==========================================
print("3. 正在组装 LCEL 问答管道...")
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# ==========================================
# 4. 针对新数据源执行测试
# ==========================================
if __name__ == "__main__":
    print("\n=== Web-RAG 系统已上线 ===")
    
    # 提出针对网页内容的新问题：最大的 Llama 3 模型有多大？
    test_question = "What's the size of the largest Llama 3 model?"
    print(f"\n🙋 提问: {test_question}")
    
    response = rag_chain.invoke(test_question)
    # AI 成功从实时网页中抓取到了答案：超过 4000 亿个参数
    print(f"🤖 回答: {response}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


1. 正在初始化数据摄取...
2. 正在进行向量化并存入数据库...
3. 正在组装 LCEL 问答管道...

=== Web-RAG 系统已上线 ===

🙋 提问: What's the size of the largest Llama 3 model?
🤖 回答: The largest Llama 3 model will have over 400 billion parameters.
